In [1]:
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input, LeakyReLU, Conv2D as conv2D, Dropout, Add, UpSampling2D, concatenate
import cv2
import random
import os
import PIL
from PIL import Image, ImageOps
from matplotlib import colors, cm
from ImprovedUnet import improved_unet


HEIGHT = 256
WIDTH = 256
TRAIN_SIZE = 1815
TEST_SIZE = 389
VALI_SIZE = 389

In [2]:
# Given prediction images and ground truth imgaes. Return the dice similarity coefficient
def dice_coef(predictions, truth, axis=(1,2,3)):
    predictions = tf.convert_to_tensor(predictions, np.float32)
    truth = tf.convert_to_tensor(truth, np.float32)
    numerator = (2.0 * (tf.reduce_sum(predictions * truth, axis=axis))) + 1
    denominator = tf.reduce_sum(predictions, axis=axis) + tf.reduce_sum(truth, axis=axis) + 1
    coef = tf.reduce_mean(numerator / denominator)
    return coef

# Return the dice similarity coefficient loss
def dice_coef_loss(predictions, truth):
    return 1 - dice_coef(predictions, truth)

# Read images from row image file name and ground truth file name
def read_images(image_file, mask_file):
    img = cv2.imread(image_file)
    img = cv2.resize(img, (HEIGHT, WIDTH))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255
    img = (img - img.mean())/img.std()
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (HEIGHT, WIDTH))
    mask = np.round(mask/255)
    mask = mask[:, :, np.newaxis]
    return img, mask

# Image gererator loaded batchsize number of random pairs of row images and ground truth images.
def image_generator(image_fnames, mask_fnames, batchsize):
    count = len(image_fnames)
    while True:
        random_indices = random.sample(range(count), batchsize)
        batch_img, batch_mask = [], []
        for index in random_indices:
            img, mask = read_images(image_fnames[index], mask_fnames[index])
            batch_img.append(img)
            batch_mask.append(mask)
        images = np.stack(batch_img, axis=0)
        masks = np.stack(batch_mask, axis=0)
        yield images, masks 

# Plot five pair of row images and ground truth images from data generator.
def plot_gen(data_gen):
    images, masks = next(data_gen)
    plot_images(images, 3)
    plot_images(masks, 1)

# Plot five images in one row. Set dim=1 when plotting gray images, set dim to 3 when plotting color images.
def plot_images(images, dim):
    fig, axs = plt.subplots(1, 5, sharey=True, figsize=(15,15))
    for i in range(5):
        if dim == 3:
            img = images[i, :, :, :]
            norm = colors.LogNorm(img.mean() + 0.5 * img.std(), img.max(), clip='True')  
            axs[i].imshow(img, cmap=cm.gray, norm=norm, origin="lower")
            axs[i].axis('off')
        elif dim == 1:
            img = images[i, :, :, 0]
            axs[i].imshow(img, cmap='gray')
            axs[i].axis('off')            
    plt.show()


# Plot five pairs of row images, ground truth images and their corresponding predictions.
def plot_comparison(images, test_masks, predictions):
    images, test_masks, predictions = generate_random_images(images, test_masks, predictions)
    plot_images(images, 3)
    plot_images(test_masks, 1)    
    plot_images(predictions, 1)

# Generate five random images for row images, ground truth images and their corresponding predictions.
def generate_random_images(test_images, test_masks, predictions):
    random_indices = random.sample(range(TEST_SIZE), 5)
    imgs = []
    preds = []
    masks = []
    for i in random_indices:
        imgs.append(test_images[i, :, :, :])
        masks.append(test_masks[i, :, :, :])
        preds.append(predictions[i, :, :, :])
    imgs = np.stack(imgs, axis=0)
    masks = np.stack(masks, axis=0)
    preds = np.stack(preds, axis=0)
    return imgs, masks, preds

# Load images from raw images and ground truth file names, return them as numpy arrays
def load_images_from_fn(image_fnames, mask_fnames):
    images = []
    masks = []
    for i in range(len(image_fnames)):
        img, mask = read_images(image_fnames[i], mask_fnames[i])
        images.append(img)
        masks.append(mask)
    images = np.stack(images, axis=0)
    masks = np.stack(masks, axis=0)
    return images, masks

# Load raw images using PIL
def load_original_test_images(test_x):
    images = []
    for file in test_x:
        img = PIL.Image.open(str(file))
        img = ImageOps.flip(img)
        img = img.resize((HEIGHT, WIDTH))
        img = np.array(img.getdata()).reshape(HEIGHT, WIDTH, 3)
        images.append(img)
    images = np.stack(images, axis=0)
    return images

In [3]:
# Load images, return training, validation generator, return test_masks, test_images in numpy format.
# return test_images_forplot because the test_images loaded are normalized. 
def load_data():
    input_images = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/input/*.jpg"))
    output_masks = sorted(glob.glob("H:\\COMP3710/report/ISIC2018_Task1-2_Training_Data/truth/*.png"))
    
    # Separate file names to train, validation and test.
    DATASET_SIZE = len(input_images)
    train_size = int(DATASET_SIZE * 0.7)
    val_size = test_size = int(DATASET_SIZE * 0.15)
    train_x = input_images[0:train_size]
    train_y = output_masks[0:train_size]
    test_x = input_images[train_size:train_size+test_size]
    test_y = output_masks[train_size:train_size+test_size]
    val_x = input_images[train_size+test_size:]
    val_y = output_masks[train_size+test_size:]
    
    train_gen = image_generator(train_x, train_y, 10)
    val_gen = image_generator(val_x, val_y, 10)
    test_images, test_masks = load_images_from_fn(test_x, test_y)
    test_images_forplot = load_original_test_images(test_x)
    print('Loading success')
    return train_gen, val_gen, test_masks, test_images, test_images_forplot

# Compile and fit model, return model and history.
def compile_model(train_gen, val_gen):
    model = improved_unet(HEIGHT, WIDTH)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef, 'acc'])
    # Create check point
    checkpoint_filepath = '/tmp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='acc',
        mode='max',
        save_best_only=True)
    # Compile model
    train_history = model.fit(train_gen, epochs=250, steps_per_epoch=10, validation_data = val_gen, validation_steps=10)
    return model, train_history

# Training loss, accuracy, and dsc are plotted.
def plot_training_results(train_history):
    plt.plot(train_history.history['loss'], 'k', label='Training loss')
    plt.plot(train_history.history['val_loss'], 'b', label = 'Validation loss')
    plt.title('Loss of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    plt.figure()
    plt.plot(train_history.history['acc'], 'k', label='Training accuracy')
    plt.plot(train_history.history['val_acc'], 'b', label='Validation accuracy')
    plt.title('Accuracy of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()

    plt.figure()
    plt.plot(train_history.history['dice_coef'], 'k', label='Training dsc ')
    plt.plot(train_history.history['val_dice_coef'], 'b', label='Validation dsc')
    plt.title('Dice similarity coefficient of trained data')
    plt.xlabel('Iterations')
    plt.ylabel('Dice similarity coefficient')
    plt.legend()
    plt.grid()


In [ ]:
def main():
    # Load training ang validation data generator, load test images and masks as numpy array. 
    # Raw test images with no preprocess is also loaded for plotting
    train_gen, val_gen, test_masks, test_images, test_images_forplot = load_data()
    # Model is compiled with given train and validation data generator.
    model, train_history = compile_model(train_gen, val_gen)
    # Training loss, accuracy, dsc are plotted.
    plot_training_results(train_history)
    # Predicted images are loaded
    predictions = np.round(model.predict(test_images))
    # Prediced images are compaired with raw image and true masks
    plot_comparison(test_images_forplot, test_masks, predictions)
    # Dice coefficient of the prediction is calculated.
    dsc = dice_coef(predictions, test_masks)
    dsc = dsc.numpy()
    print('Dice similarity coefficient: ' + str(dsc))

if __name__ == "__main__":
    main()

Loading success
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 10 steps
Epoch 1/250
10/10 [==============================] - 17s 2s/step - loss: 0.6989 - dice_coef: 0.2438 - acc: 0.6562 - val_loss: 0.6897 - val_dice_coef: 0.2682 - val_acc: 0.7574
Epoch 2/250
10/10 [==============================] - 14s 1s/step - loss: 0.6885 - dice_coef: 0.2534 - acc: 0.7846 - val_loss: 0.6882 - val_dice_coef: 0.3041 - val_acc: 0.7238
Epoch 3/250
10/10 [==============================] - 14s 1s/step - loss: 0.6823 - dice_coef: 0.2382 - acc: 0.8018 - val_loss: 0.6776 - val_dice_coef: 0.2955 - val_acc: 0.7269
Epoch 4/250
10/10 [==============================] - 13s 1s/step - loss: 0.6153 - dice_coef: 0.2091 - acc: 0.7799 - val_loss: 0.5783 - val_dice_coef: 0.2335 - val_acc: 0.7308
Epoch 5/250
10/10 [==============================] - 13s 1s/step - loss: 0.5291 - dice_coef: 0.1538 - acc: 0.8165 - val_loss: 0.5528 - val_dice_coef: 0.2584 - val_acc: 0.7248
Epoch 6/250
10/10

10/10 [==============================] - 13s 1s/step - loss: 0.2301 - dice_coef: 0.5901 - acc: 0.9137 - val_loss: 0.3301 - val_dice_coef: 0.6118 - val_acc: 0.8798
Epoch 47/250
10/10 [==============================] - 13s 1s/step - loss: 0.2817 - dice_coef: 0.5146 - acc: 0.9065 - val_loss: 0.2534 - val_dice_coef: 0.6126 - val_acc: 0.9241
Epoch 48/250
10/10 [==============================] - 13s 1s/step - loss: 0.2248 - dice_coef: 0.6302 - acc: 0.9148 - val_loss: 0.3338 - val_dice_coef: 0.5791 - val_acc: 0.8982
Epoch 49/250
10/10 [==============================] - 14s 1s/step - loss: 0.2599 - dice_coef: 0.6127 - acc: 0.9055 - val_loss: 0.2662 - val_dice_coef: 0.6074 - val_acc: 0.9111
Epoch 50/250
10/10 [==============================] - 15s 1s/step - loss: 0.2780 - dice_coef: 0.5294 - acc: 0.9017 - val_loss: 0.3018 - val_dice_coef: 0.5545 - val_acc: 0.9016
Epoch 51/250
10/10 [==============================] - 14s 1s/step - loss: 0.2259 - dice_coef: 0.5879 - acc: 0.9189 - val_loss: 0.2701